In [1]:
import numpy as np
import pandas as pd
from IPython.display import display
import matplotlib.pyplot as plt
%matplotlib inline

import seaborn as sns

In [2]:
df = pd.read_csv('bank_transactions.csv')
df

,TransactionID,CustomerID,CustomerDOB,CustGender,CustLocation,CustAccountBalance,TransactionDate,TransactionTime,TransactionAmount (INR)
0,T1,C5841053,10/1/94,F,JAMSHEDPUR,17819.05,2/8/16,143207,25.0
1,T2,C2142763,4/4/57,M,JHAJJAR,2270.69,2/8/16,141858,27999.0
2,T3,C4417068,26/11/96,F,MUMBAI,17874.44,2/8/16,142712,459.0
3,T4,C5342380,14/9/73,F,MUMBAI,866503.21,2/8/16,142714,2060.0
4,T5,C9031234,24/3/88,F,NAVI MUMBAI,6714.43,2/8/16,181156,1762.5
...,...,...,...,...,...,...,...,...,...
1048562,T1048563,C8020229,8/4/90,M,NEW DELHI,7635.19,18/9/16,184824,799.0
1048563,T1048564,C6459278,20/2/92,M,NASHIK,27311.42,18/9/16,183734,460.0
1048564,T1048565,C6412354,18/5/89,M,HYDERABAD,221757.06,18/9/16,183313,770.0
1048565,T1048566,C6420483,30/8/78,M,VISAKHAPATNAM,10117.87,18/9/16,184706,1000.0


In [3]:
null_columns = df.columns[df.isnull().any()]
null_dob = df[df['CustomerDOB'].isnull()]
null_gender = df[df['CustGender'].isnull()]
null_location = df[df['CustLocation'].isnull()]
null_balance = df[df['CustAccountBalance'].isnull()]
print("columns with null values: ", null_columns)
print("Date of Birth null shape: ", null_dob.shape)
print("gender null shape: ", null_gender.shape)
print("location null shape: ", null_location.shape)
print("balance null shape: ", null_balance.shape)

columns with null values:  Index(['CustomerDOB', 'CustGender', 'CustLocation', 'CustAccountBalance'], dtype='object')
Date of Birth null shape:  (3397, 9)
gender null shape:  (1100, 9)
location null shape:  (151, 9)
balance null shape:  (2369, 9)


In [4]:
mode_gender = df['CustGender'].mode()[0]
mode_location = df['CustLocation'].mode()[0]
mean_balance = df['CustAccountBalance'].mean()
df = df.dropna(subset='CustomerDOB')
df['CustGender'].fillna(mode_gender, inplace=True)
df['CustLocation'].fillna(mode_location, inplace=True)
df['CustAccountBalance'].fillna(mean_balance, inplace=True)
df = df.drop(df[df['CustomerDOB'] == '1/1/1800'].index)

C:\Users\irfan_j5mtdic\AppData\Local\Temp\ipykernel_37648\930001085.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['CustGender'].fillna(mode_gender, inplace=True)
C:\Users\irfan_j5mtdic\AppData\Local\Temp\ipykernel_37648\930001085.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['CustLocation'].fillna(mode_location, inplace=True)
C:\Users\irfan_j5mtdic\AppData\Local\Temp\ipykernel_37648\930001085.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#return

In [5]:
df['TransactionTime'] = df['TransactionTime'].astype(str)


In [288]:
df

,TransactionID,CustomerID,CustomerDOB,CustGender,CustLocation,CustAccountBalance,TransactionDate,TransactionTime,TransactionAmount (INR)
0,T1,C5841053,10/1/94,F,JAMSHEDPUR,17819.05,2/8/16,143207,25.0
1,T2,C2142763,4/4/57,M,JHAJJAR,2270.69,2/8/16,141858,27999.0
2,T3,C4417068,26/11/96,F,MUMBAI,17874.44,2/8/16,142712,459.0
3,T4,C5342380,14/9/73,F,MUMBAI,866503.21,2/8/16,142714,2060.0
4,T5,C9031234,24/3/88,F,NAVI MUMBAI,6714.43,2/8/16,181156,1762.5
...,...,...,...,...,...,...,...,...,...
1048562,T1048563,C8020229,8/4/90,M,NEW DELHI,7635.19,18/9/16,184824,799.0
1048563,T1048564,C6459278,20/2/92,M,NASHIK,27311.42,18/9/16,183734,460.0
1048564,T1048565,C6412354,18/5/89,M,HYDERABAD,221757.06,18/9/16,183313,770.0
1048565,T1048566,C6420483,30/8/78,M,VISAKHAPATNAM,10117.87,18/9/16,184706,1000.0


In [8]:
def convert_and_format_dates_and_times(df):

    if not pd.api.types.is_datetime64_any_dtype(df['CustomerDOB']):
        df['CustomerDOB'] = pd.to_datetime(df['CustomerDOB'].str[:-2] + '19' + df['CustomerDOB'].str[-2:], format='%d/%m/%Y', errors='coerce')
    
    # Check if TransactionDate is not already datetime and convert if necessary
    if not pd.api.types.is_datetime64_any_dtype(df['TransactionDate']):
        df['TransactionDate'] = pd.to_datetime(df['TransactionDate'].str[:-2] + '20' + df['TransactionDate'].str[-2:], format='%d/%m/%Y', errors='coerce')
    
    # Convert TransactionTime to datetime and then extract the time
        if not pd.api.types.is_datetime64_any_dtype(df['TransactionTime']):
        # Check if the dtype is not timedelta (which is not handled by is_time_dtype)
            if not pd.api.types.is_timedelta64_dtype(df['TransactionTime']):
                df['TransactionTime'] = pd.to_datetime(df['TransactionTime'].str[:-4] + ":" + df['TransactionTime'].str[-4:-2] + ":" + df['TransactionTime'].str[-2:], format='%H:%M:%S', errors='coerce')
                df['TransactionTime'] = df['TransactionTime'].dt.time
    
    return df

# Example usage:
# Assuming you have a DataFrame named `df` with the columns 'CustomerDOB', 'TransactionDate', and 'TransactionTime'
df = convert_and_format_dates_and_times(df)

In [9]:
df

,TransactionID,CustomerID,CustomerDOB,CustGender,CustLocation,CustAccountBalance,TransactionDate,TransactionTime,TransactionAmount (INR)
0,T1,C5841053,1994-01-10,F,JAMSHEDPUR,17819.05,2016-08-02,14:32:07,25.0
1,T2,C2142763,1957-04-04,M,JHAJJAR,2270.69,2016-08-02,14:18:58,27999.0
2,T3,C4417068,1996-11-26,F,MUMBAI,17874.44,2016-08-02,14:27:12,459.0
3,T4,C5342380,1973-09-14,F,MUMBAI,866503.21,2016-08-02,14:27:14,2060.0
4,T5,C9031234,1988-03-24,F,NAVI MUMBAI,6714.43,2016-08-02,18:11:56,1762.5
...,...,...,...,...,...,...,...,...,...
1048562,T1048563,C8020229,1990-04-08,M,NEW DELHI,7635.19,2016-09-18,18:48:24,799.0
1048563,T1048564,C6459278,1992-02-20,M,NASHIK,27311.42,2016-09-18,18:37:34,460.0
1048564,T1048565,C6412354,1989-05-18,M,HYDERABAD,221757.06,2016-09-18,18:33:13,770.0
1048565,T1048566,C6420483,1978-08-30,M,VISAKHAPATNAM,10117.87,2016-09-18,18:47:06,1000.0


In [10]:
df_cleaned = df.copy()

In [134]:
df_cleaned['CustGender'].value_counts()

CustGender
M    714283
F    273548
Name: count, dtype: int64